In [1]:
import numpy as np
import cv2
import shutil
import os
from xml.dom.minidom import Document    

def widerFaceToRoidb(txtFileName):
    fp = open(txtFileName,'r')
    datas = fp.readlines()
    faceData = []
    print('datas number is: ', len(datas))
    faceCount = 0
    i = 0
    boxCount = 0
    while( i < len(datas)):
        imagePath = datas[i].strip()
        boxesNumber = int(datas[i+1].strip())
        boxes = []
        boxes.append(imagePath)
        # print('face Count :%s,'%(faceCount))
        # print('facefileName:%s'%imagePath)
        #print('boxesNumber:%d'%boxesNumber)
        i += 1
        for k in range(boxesNumber):
            i = i + 1
            boxCount += 1
            box = datas[i].strip().split(' ')
            #print('%d: box:%s\n'%(i,box))
            boxes.append(box)
        i += 1
        faceCount += 1
        faceData.append(boxes)
    print('fileCount:%d,boxCount:%d'%(faceCount, boxCount))
    return faceData
        
def checkWay(way):
    if not os.path.exists(way):
        os.makedirs(way)
def setJpeg(roidb,imageFolder,vocJpegWay, vocImageSetsMainWay,mode):
    imageName =  roidb[0]
    imagePath = imageFolder+imageName
     # copy -> vocJpegWay
    imageName = imageName.split('/')[1]
    
    shutil.copy(imagePath,vocJpegWay+imageName)
    outFile= open(vocImageSetsMainWay+mode+'.txt','a')
    outFile.write(imageName.split('.')[0]+'\n')
    outFile.close()
   
    

def writeXml(roidb, imageFolder,vocXmlWay,vocJpegWay):
    erroFile = open('erroFile.txt','a')
    imageName =  roidb[0]
    labelData = np.array(roidb[1:],dtype=int).reshape(1,-1)
    
    bboxes = []
    classId = []
    otherDatas = []
    imagePath = imageFolder+imageName
    print('imagePath:%s\n'%imagePath)
    img = cv2.imread(imagePath)
    h = img.shape[0]
    w = img.shape[1]
                                
    imageName = imageName.split('/')[1]
                                
    for i in range(0,labelData.shape[1],10) :
         box = labelData[0][i:i+4]
         box[2] = box[0] + box[2]
         box[3] = box[1] + box[3]                     
         for j in range(4):
            if box[j] < 1:
                box[j] = 1
         for j in range(0,4,2):
            if box[j] > w:
                box[j] = w
         for j in range(1,4,2):
            if box[j] > h:
                box[j] = h
         # assert box[2]>=box[0], 'box[2]>=box[0]'
         # assert box[3]>=box[1], 'box[3]>=box[1]'
#          assert box[0]>=1,'x1<0'
#          assert box[1]>=1,'y1<0'
#          assert box[2]>=1,'x2<0'
#          assert box[3]>=1,'y2<0'
#          assert box[0]<=w,'x1>w'
#          assert box[1]<=h,'y1>h'
#          assert box[2]<=w,'x2>w'
#          assert box[3]<=h,'y2>h'
         if box[2] < box[0] or box[3] <box[1]:
             erroFile.write(imagePath+'\n')
             continue
         bboxes.append(box)
         className = 'face'
         classId.append(className)
         otherDatas.append(labelData[0][i+4:i+10])  
          
    
    folder_txt=vocJpegWay.split('/')[-2]
    doc=Document()
    annotation=doc.createElement('annotation')
    doc.appendChild(annotation)

    #folder
    folder = doc.createElement('folder')
    folder_txt = doc.createTextNode(folder_txt)
    folder.appendChild(folder_txt)
    annotation.appendChild(folder)

    #filename
    filename = doc.createElement('filename')
    filename_txt = doc.createTextNode(imageName.split('.')[0])
    filename.appendChild(filename_txt)
    annotation.appendChild(filename)

    #path
    path = doc.createElement('path')
    path_txt = doc.createTextNode(vocJpegWay+imageName)
    path.appendChild(path_txt)
    annotation.appendChild(path)

    #size
    size = doc.createElement('size')
    annotation.appendChild(size)

    #width
    width = doc.createElement('width')
    width_txt = doc.createTextNode(str(w))
    width.appendChild(width_txt)
    size.appendChild(width)

    #height
    height = doc.createElement('height')
    height_txt = doc.createTextNode(str(h))
    height.appendChild(height_txt)
    size.appendChild(height)

    #depth
    depth = doc.createElement('depth')
    depth_txt = doc.createTextNode('3')
    depth.appendChild(depth_txt)
    size.appendChild(depth)

    #segmented
    segmented = doc.createElement('segmented')
    segmented_txt = doc.createTextNode('0')
    segmented.appendChild(segmented_txt)
    annotation.appendChild(folder)
    object1=[]
    for i in range(len(bboxes)):
        object1.append('name'+str(i))
        object1[i]= doc.createElement('object')
        annotation.appendChild(object1[i])

        #name
        name = doc.createElement('name')
        name_txt = doc.createTextNode(classId[i])
        name.appendChild(name_txt)
        object1[i].appendChild(name)
        #truncated
        truncated = doc.createElement('truncated')
        truncated_txt = doc.createTextNode('0')
        truncated.appendChild(truncated_txt)
        object1[i].appendChild(truncated)
        #difficult
        difficult = doc.createElement('difficult')
        difficult_txt = doc.createTextNode('0')
        difficult.appendChild(difficult_txt)
        object1[i].appendChild(difficult)
        # blur
        blur = doc.createElement('blur')
        blur_txt = doc.createTextNode(str(otherDatas[i][0]))
        blur.appendChild(blur_txt)
        object1[i].appendChild(blur)
        # expression
        expression = doc.createElement('expression')
        expression_txt = doc.createTextNode(str(otherDatas[i][1]))
        expression.appendChild(expression_txt)
        object1[i].appendChild(expression)
        # illumination
        illumination = doc.createElement('illumination')
        illumination_txt = doc.createTextNode(str(otherDatas[i][2]))
        illumination.appendChild(illumination_txt)
        object1[i].appendChild(illumination)
        # invalid
        invalid = doc.createElement('invalid')
        invalid_txt = doc.createTextNode(str(otherDatas[i][3]))
        invalid.appendChild(invalid_txt)
        object1[i].appendChild(invalid)
        # occlusion
        occlusion = doc.createElement('occlusion')
        occlusion_txt = doc.createTextNode(str(otherDatas[i][4]))
        occlusion.appendChild(occlusion_txt)
        object1[i].appendChild(occlusion)
        #pose
        pose = doc.createElement('pose')
        pose_txt = doc.createTextNode(str(otherDatas[i][5]))
        pose.appendChild(pose_txt)
        object1[i].appendChild(pose)
        
        #bndbox
        bndbox = doc.createElement('bndbox')
        object1[i].appendChild(bndbox)

        #xmin
        xmin = doc.createElement('xmin')
        xmin_txt = doc.createTextNode(str(int(bboxes[i][0])))
        xmin.appendChild(xmin_txt)
        bndbox.appendChild(xmin)

        #ymin
        ymin = doc.createElement('ymin')
        ymin_txt = doc.createTextNode(str(int(bboxes[i][1])))
        ymin.appendChild(ymin_txt)
        bndbox.appendChild(ymin)

        #xmax
        xmax = doc.createElement('xmax')
        xmax_txt = doc.createTextNode(str(int(bboxes[i][2])))
        xmax.appendChild(xmax_txt)
        bndbox.appendChild(xmax)

        #ymax
        ymax = doc.createElement('ymax')
        ymax_txt = doc.createTextNode(str(int(bboxes[i][3])))
        ymax.appendChild(ymax_txt)
        bndbox.appendChild(ymax)
    xmlName=vocXmlWay+imageName.split('.')[0]+'.xml'
    f=open(xmlName,'w')
    doc.writexml(f,indent = '\t',newl = '\n', addindent = '\t',encoding='utf-8')
    f.close()
def makeAll( vocXmlWay,
             vocJpegWay,
             vocImageSetsMainWay,
             labelTxtPath,
             imageFolder,
             mode
            ):
    roidbs = widerFaceToRoidb(labelTxtPath)
    cnt = 1
    for roidb in roidbs:
        print(cnt,':processed--->',roidb[0])
        writeXml(roidb, imageFolder,vocXmlWay,vocJpegWay)
        setJpeg(roidb,imageFolder,vocJpegWay, vocImageSetsMainWay,mode)
        cnt += 1

if  __name__ == '__main__':
    print('hello widerface')
    vocWay='../VOCdevkit/VOC2007-test' # VOC FILE folder
    vocXmlWay=vocWay+'/Annotations/' # Annotations File folder
    vocJpegWay=vocWay+'/JPEGImages/' # image File
    vocImageSetsMainWay = vocWay+'/ImageSets/Main/' # txt file
         
    mode = 'train'  # data's mode (train/val)            
    labelTxtPath = '../widerface/wider_face_split/wider_face_%s_bbx_gt.txt'%(mode) # widerface Annotations file  
    imageFolder = '../widerface/WIDER_%s/images/'%(mode)                           # widerface image File 

    checkWay(vocWay)
    checkWay(vocJpegWay)
    checkWay(vocImageSetsMainWay)
    checkWay(vocXmlWay)
    makeAll( vocXmlWay,
             vocJpegWay,
             vocImageSetsMainWay,
             labelTxtPath,
             imageFolder,
             mode
            )

hello widerface
('datas number is: ', 185184)
fileCount:12880,boxCount:159424
(1, ':processed--->', '0--Parade/0_Parade_marchingband_1_849.jpg')
imagePath:../widerface/WIDER_train/images/0--Parade/0_Parade_marchingband_1_849.jpg

(2, ':processed--->', '0--Parade/0_Parade_Parade_0_904.jpg')
imagePath:../widerface/WIDER_train/images/0--Parade/0_Parade_Parade_0_904.jpg

(3, ':processed--->', '0--Parade/0_Parade_marchingband_1_799.jpg')
imagePath:../widerface/WIDER_train/images/0--Parade/0_Parade_marchingband_1_799.jpg

(4, ':processed--->', '0--Parade/0_Parade_marchingband_1_117.jpg')
imagePath:../widerface/WIDER_train/images/0--Parade/0_Parade_marchingband_1_117.jpg

(5, ':processed--->', '0--Parade/0_Parade_marchingband_1_778.jpg')
imagePath:../widerface/WIDER_train/images/0--Parade/0_Parade_marchingband_1_778.jpg

(6, ':processed--->', '0--Parade/0_Parade_Parade_0_343.jpg')
imagePath:../widerface/WIDER_train/images/0--Parade/0_Parade_Parade_0_343.jpg

(7, ':processed--->', '0--Parade/0

(57, ':processed--->', '0--Parade/0_Parade_Parade_0_676.jpg')
imagePath:../widerface/WIDER_train/images/0--Parade/0_Parade_Parade_0_676.jpg

(58, ':processed--->', '0--Parade/0_Parade_Parade_0_27.jpg')
imagePath:../widerface/WIDER_train/images/0--Parade/0_Parade_Parade_0_27.jpg

(59, ':processed--->', '0--Parade/0_Parade_Parade_0_359.jpg')
imagePath:../widerface/WIDER_train/images/0--Parade/0_Parade_Parade_0_359.jpg

(60, ':processed--->', '0--Parade/0_Parade_Parade_0_532.jpg')
imagePath:../widerface/WIDER_train/images/0--Parade/0_Parade_Parade_0_532.jpg

(61, ':processed--->', '0--Parade/0_Parade_marchingband_1_1046.jpg')
imagePath:../widerface/WIDER_train/images/0--Parade/0_Parade_marchingband_1_1046.jpg

(62, ':processed--->', '0--Parade/0_Parade_Parade_0_92.jpg')
imagePath:../widerface/WIDER_train/images/0--Parade/0_Parade_Parade_0_92.jpg

(63, ':processed--->', '0--Parade/0_Parade_marchingband_1_472.jpg')
imagePath:../widerface/WIDER_train/images/0--Parade/0_Parade_marchingband_1_

(117, ':processed--->', '0--Parade/0_Parade_Parade_0_868.jpg')
imagePath:../widerface/WIDER_train/images/0--Parade/0_Parade_Parade_0_868.jpg

(118, ':processed--->', '0--Parade/0_Parade_marchingband_1_163.jpg')
imagePath:../widerface/WIDER_train/images/0--Parade/0_Parade_marchingband_1_163.jpg

(119, ':processed--->', '0--Parade/0_Parade_Parade_0_176.jpg')
imagePath:../widerface/WIDER_train/images/0--Parade/0_Parade_Parade_0_176.jpg

(120, ':processed--->', '0--Parade/0_Parade_marchingband_1_388.jpg')
imagePath:../widerface/WIDER_train/images/0--Parade/0_Parade_marchingband_1_388.jpg

(121, ':processed--->', '0--Parade/0_Parade_Parade_0_803.jpg')
imagePath:../widerface/WIDER_train/images/0--Parade/0_Parade_Parade_0_803.jpg

(122, ':processed--->', '0--Parade/0_Parade_Parade_0_4.jpg')
imagePath:../widerface/WIDER_train/images/0--Parade/0_Parade_Parade_0_4.jpg

(123, ':processed--->', '0--Parade/0_Parade_marchingband_1_473.jpg')
imagePath:../widerface/WIDER_train/images/0--Parade/0_Parad

KeyboardInterrupt: 